In [26]:
# export the model
#
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
import joblib
from sklearn.preprocessing import LabelEncoder

In [27]:
# import csvs from github
url_us = "https://raw.githubusercontent.com/James-yeh/Spotify-Rec/main/US.csv"
url_tracks = "https://media.githubusercontent.com/media/James-yeh/Spotify-Rec/main/tracks.csv"
url_datao = "https://raw.githubusercontent.com/James-yeh/Spotify-Rec/main/data_o.csv"
url_dbgo = "https://raw.githubusercontent.com/James-yeh/Spotify-Rec/main/data_by_genres_o.csv"

In [28]:
# tracks = pd.read_csv('tracks.csv')
# data_o = pd.read_csv('data_o.csv')
# genres = pd.read_csv('data_by_genres_o.csv')
# df = pd.read_csv('US.csv')
tracks = pd.read_csv(url_tracks)
data_o = pd.read_csv(url_datao)
genres = pd.read_csv(url_dbgo)
df = pd.read_csv(url_us)
df['release_date'] = pd.to_datetime(df['release_date'])
df.drop(columns=['id','id_artists'],inplace=True)
df['year'] = pd.DatetimeIndex(df['release_date']).year
df.drop(columns=['release_date'],inplace = True)

In [29]:
df

,name,popularity,duration_ms,explicit,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year
0,Carve,6,126903,0,['Uli'],0.645,0.4450,0,-13.338,1,0.4510,0.67400,0.744000,0.1510,0.127,104.851,3,1922
1,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],0.695,0.2630,0,-22.136,1,0.9570,0.79700,0.000000,0.1480,0.655,102.009,1,1922
2,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],0.434,0.1770,1,-21.180,1,0.0512,0.99400,0.021800,0.2120,0.457,130.418,5,1922
3,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],0.321,0.0946,7,-27.961,1,0.0504,0.99500,0.918000,0.1040,0.397,169.980,3,1922
4,Lady of the Evening,0,163080,0,['Dick Haymes'],0.402,0.1580,3,-16.900,0,0.0390,0.98900,0.130000,0.3110,0.196,103.220,4,1922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93130,Easter In Miami,64,187097,1,['Kodak Black'],0.746,0.8290,7,-5.124,0,0.3230,0.10500,0.000000,0.1450,0.681,93.006,4,2021
93131,Thugged Out (feat. Kodak Black),71,185992,1,"['YNW Melly', 'Kodak Black']",0.720,0.5970,1,-6.501,1,0.2170,0.17100,0.000000,0.1700,0.367,150.138,4,2021
93132,Créeme,2,214293,0,"['KAROL G', 'Maluma']",0.714,0.8740,6,-3.576,1,0.0549,0.12800,0.000146,0.1050,0.648,96.998,4,2021
93133,Punto G,4,183160,0,['KAROL G'],0.791,0.7460,6,-4.716,1,0.0907,0.00938,0.000125,0.1250,0.903,165.994,4,2021


In [30]:
# remove duplicates
print("Origional:", df.shape)
df = df.drop_duplicates()
print("Cleaned:", df.shape)

# store names and remove them
names = df['name']

# label encode artists so It can select similar artists
enc = LabelEncoder()
df['artists'] = enc.fit_transform(df['artists'])

Origional: (93135, 18)
Cleaned: (92767, 18)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [31]:
X = df.drop(columns=['name']).values

In [32]:
neighbors = 20
nn = NearestNeighbors(n_neighbors=neighbors + 1)
nn.fit(X)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=21, p=2,
                 radius=1.0)

In [33]:
# returns the nearest neighbor number of nearest songs given a song object
def song_suggester(song_obj):
    distance, neighbors = nn.kneighbors(np.array([song_obj]))
    suggestions = []
    for i in neighbors[0][1:]:
        suggestions.append([df['name'].iloc[i],df['artists'].iloc[i]])
    return suggestions

In [34]:
def song(name):
    song = 'song'
    try:
        song = song_suggester(X[df['name']==name][0])
    except:
        print("Invalid Name Retard")
    return song

In [35]:
# example for creating sililar songs
# Look at 500 it has interesting insight
example = 29483

print('Song:',df['name'].iloc[example])

suggestions = song_suggester(X[example])
sug = pd.DataFrame(suggestions,columns = ['song','artist'])
sug.head(20)


Song: Lillie


,song,artist
0,Kelo,13563
1,Mental Hopscotch,13669
2,Dint' 'o bar,13618
3,All I Need,13363
4,If We're Not Back In Love By Monday,13344
5,Smoke One,13912
6,Southern Man,13348
7,Angels Like You,13589
8,Good Morning Little School Girl,13933
9,"Look, Back & Laugh",13637


In [36]:
# filename = 'NearestNeighbor.sav'
# joblib.dump(nn, filename)

In [37]:
# df.to_csv('data.csv')